In [1]:
import os
import nltk
# nltk.download('popular')
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

from scipy.stats import randint
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


In [2]:
df = pd.read_csv('/Users/anna/220136_A2/trainingData/train.csv')
testdf=pd.read_csv('/Users/anna/220136_A2/trainingData/train.csv')
print(df.shape)

(2225, 2)


In [3]:
df['text'] = df['text'].fillna('')
testdf['text'] = testdf['text'].fillna('')

In [4]:
target=pd.DataFrame(df.category.unique()).values
target

array([['tech'],
       ['business'],
       ['sport'],
       ['entertainment'],
       ['politics']], dtype=object)

converting the letters to lower case, and tokenizing the words

In [5]:
df['text'] = [entry.lower() for entry in df['text']]
df['text']= [word_tokenize(entry) for entry in df['text']]

testdf['text'] = [entry.lower() for entry in testdf['text']]
testdf['text']= [word_tokenize(entry) for entry in testdf['text']]

In [6]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [7]:
word_Lemmatized = WordNetLemmatizer()

In [ ]:
for index, entry in enumerate(df['text']):

    Final_words = []
    for word, tag in pos_tag(entry):
#         print(word, tag)
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,pos=tag_map[tag[0]])
            Final_words.append(word_Final)
    df.loc[index,'text_final'] = str(Final_words);
    

/var/folders/sm/xwk1_yqs0dxbpmhqtxg9wjgw0000gn/T/ipykernel_42618/1410304338.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['tv', 'future', 'hand', 'viewer', 'home', 'theatre', 'system', 'plasma', 'tv', 'digital', 'video', 'recorder', 'move', 'living', 'room', 'way', 'people', 'watch', 'tv', 'radically', 'different', 'five', 'year', 'time', 'accord', 'expert', 'panel', 'gather', 'annual', 'consumer', 'electronics', 'show', 'la', 'vegas', 'discuss', 'new', 'technology', 'impact', 'one', 'favourite', 'pastime', 'u', 'lead', 'trend', 'programme', 'content', 'deliver', 'viewer', 'via', 'home', 'network', 'cable', 'satellite', 'telecom', 'company', 'broadband', 'service', 'provider', 'front', 'room', 'portable', 'device', 'one', 'technology', 'ce', 'digital', 'personal', 'video', 'recorder', 'dvr', 'pvr', 'box', 'like', 'u', 'tivo', 'uk', 'system', 'allow', 'people', 'record', 'store', 'play', 'pause', 'forward', '

In [ ]:
for index, entry in enumerate(testdf['text']):

    Final_words = []
    for word, tag in pos_tag(entry):
#         print(word, tag)
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,pos=tag_map[tag[0]])
            Final_words.append(word_Final)
    testdf.loc[index,'text_final'] = str(Final_words);

In [12]:
Train_X=df['text_final']
Test_X=testdf['text_final']
Train_Y=df['category']
Test_Y=testdf['category']

# from sklearn.model_selection import train_test_split

# Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['text_final'],df['category'],test_size=0.3)


In [13]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [16]:
df['text_final'] = df['text_final'].fillna('')
testdf['text_final'] = df['text_final'].fillna('')

In [17]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df['text_final'])
Tfidf_vect.fit(testdf['text_final'])

TfidfVectorizer(max_features=5000)

In [18]:
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)
Train_X_Tfidf

<2225x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 271252 stored elements in Compressed Sparse Row format>

In [19]:
NB = naive_bayes.MultinomialNB()
NB.fit(Train_X_Tfidf,Train_Y)

MultinomialNB()

In [20]:
SVM = svm.SVC(C = 2.0, kernel = 'linear', gamma = 'auto')
SVM.fit(Train_X_Tfidf, Train_Y)

SVC(C=2.0, gamma='auto', kernel='linear')

In [21]:
# predict the labels on validation dataset
pred_NB = NB.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(pred_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  98.78651685393258


In [22]:
print(classification_report(Test_Y, pred_NB))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99       510
           1       0.99      0.98      0.99       386
           2       0.98      0.99      0.99       417
           3       1.00      1.00      1.00       511
           4       0.98      0.99      0.98       401

    accuracy                           0.99      2225
   macro avg       0.99      0.99      0.99      2225
weighted avg       0.99      0.99      0.99      2225



In [23]:
pred_SVM = SVM.predict(Test_X_Tfidf)

print("SVM Accuracy Score -> ",accuracy_score(pred_SVM, Test_Y)*100)

SVM Accuracy Score ->  99.95505617977528


In [24]:
print(classification_report(Test_Y, pred_SVM))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       510
           1       1.00      1.00      1.00       386
           2       1.00      1.00      1.00       417
           3       1.00      1.00      1.00       511
           4       1.00      1.00      1.00       401

    accuracy                           1.00      2225
   macro avg       1.00      1.00      1.00      2225
weighted avg       1.00      1.00      1.00      2225

